<a href="https://colab.research.google.com/github/minjae0501/yolo_block/blob/master/lstm_yolo_detecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!git clone https://github.com/minjae0501/yolo_block.git

Cloning into 'yolo_block'...
remote: Enumerating objects: 3012, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3012 (delta 3), reused 13 (delta 1), pack-reused 2993
Receiving objects: 100% (3012/3012), 117.88 MiB | 50.70 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
%pip install ultralytics
%pip install mediapipe
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.237 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.6/166.8 GB disk)


In [3]:

import cv2
import mediapipe as mp
from ultralytics import YOLO
import numpy as np

In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [5]:
import torch

if torch.cuda.is_available() == True:
    device = 'cuda:0'
else:
    device = 'cpu'

print(device)

cuda:0


In [6]:
class hand_LSTM(nn.Module):
    def __init__(self):
        super(hand_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size = 71, hidden_size = 128, num_layers=1, batch_first=True) # xyz 63 + x_center, y_center = 65
        self.lstm2 = nn.LSTM(input_size = 128, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size = 256, hidden_size = 512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size = 256, hidden_size = 128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size = 128, hidden_size = 64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size = 64, hidden_size = 32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32, 2)
    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:, -1,:])
        return x

In [7]:
# 모델 불러오기
model_path = '/content/yolo_block/lstm_pth/lstm_model_01.pth'

lstm_model = hand_LSTM().to(device)
lstm_model.load_state_dict(torch.load(model_path, map_location=device))
lstm_model.eval()
print("모델이 성공적으로 불러와졌습니다.")

모델이 성공적으로 불러와졌습니다.


In [22]:
# YOLO 객체 감지 모델 초기화
best_model = '/content/yolo_block/block_best_02.pt'
# best_model = 'best.pt'
yolo_model = YOLO(best_model)

In [23]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list:
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [24]:
length = 20
interval = 1
detect_cls = 1

video_path = '/content/yolo_block/data/video_data/val/KakaoTalk_20240108_192602489.mp4'
# cv2.destroyAllWindows()
cap = cv2.VideoCapture(video_path)
img_list = []
if cap.isOpened():
    cnt = 0
    while True:
        ret , img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            if cnt == interval:
                img_list.append(img)
                cnt =0
            # cv2.imshow('test_video', img)
            # cv2.waitKey(1)
            cnt += 1

        else: break
cap.release()
# cv2.destroyAllWindows()

print('저장된 frame의 개수 {}'.format(len(img_list)))

저장된 frame의 개수 624


In [25]:
from tqdm import tqdm

lstm_model.eval()
out_img_list = []
dataset = []
status = 'None'

detect_cls = 0
hand_cls = 6

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

print('시퀀스 데이터 분석 중..')
xyz_list_list = []

for img in tqdm(img_list):
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    xyz_list = []
    filtered_boxes, filtered_box_class = [], []

    if not results.multi_hand_landmarks:
          continue

    for x_y_z in results.multi_hand_landmarks:
          for landmark in x_y_z.landmark:
            xyz_list.append(landmark.x)
            xyz_list.append(landmark.y)
            xyz_list.append(landmark.z)

          mp_drawing.draw_landmarks(img, x_y_z, mp_hands.HAND_CONNECTIONS)

    # YOLO 박스
    box_results = yolo_model.predict(img, conf = 0.6)
    box_results = box_results[0].boxes
    boxes = box_results.xyxy.cpu().tolist()
    box_class = box_results.cls.cpu().tolist()
    confs = box_results.conf.float().cpu().tolist()


    x1, y1, x2, y2 = 0, 0, 0, 0
    hx1, hy1, hx2, hy2 = 0,0,0,0
    for idx, cls in enumerate(box_class):
        if int(cls) == detect_cls:
            x1, y1, x2, y2 = boxes[idx]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        elif int(cls) == hand_cls:
            hx1, hy1, hx2, hy2 = boxes[idx]
            hx1, hy1, hx2, hy2 = int(hx1), int(hy1), int(hx2), int(hy2)
            cv2.rectangle(img, (int(hx1), int(hy1)), (int(hx2), int(hy2)), (0, 0, 255), 2)

    xyz_list.append(x1)
    xyz_list.append(y1)
    xyz_list.append(x2)
    xyz_list.append(y2)
    xyz_list.append(hx1)
    xyz_list.append(hy1)
    xyz_list.append(hx2)
    xyz_list.append(hy2)

    xyz_list_list.append(xyz_list)

    if len(xyz_list_list) == length:
        dataset = []
        dataset.append({'key': 0, 'value': xyz_list_list})
        dataset = MyDataset(dataset)
        dataset = DataLoader(dataset)
        xyz_list_list = []
        for data, label in dataset:
            data = data.to(device)
            with torch.no_grad():
                result = lstm_model(data)
                _, out = torch.max(result, 1)
                if out.item() == 0: status = 'Release'
                else: status = 'Grab'

    cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0, 225), 2)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out_img_list.append(img)

시퀀스 데이터 분석 중..


  0%|          | 0/624 [00:00<?, ?it/s]


0: 640x640 1 blue_bridge, 1 hand, 7.3ms
Speed: 1.8ms preprocess, 7.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/624 [00:00<02:07,  4.90it/s]


0: 640x640 1 blue_bridge, 1 hand, 7.4ms
Speed: 2.0ms preprocess, 7.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 7.4ms
Speed: 1.8ms preprocess, 7.4ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 7.4ms
Speed: 1.9ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


...

 99%|█████████▉| 619/624 [00:24<00:00, 25.38it/s]


0: 640x640 1 blue_bridge, 1 hand, 8.4ms
Speed: 2.4ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 8.0ms
Speed: 2.1ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 8.2ms
Speed: 1.7ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


100%|█████████▉| 622/624 [00:24<00:00, 24.34it/s]


0: 640x640 1 blue_bridge, 1 hand, 8.4ms
Speed: 2.3ms preprocess, 8.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 blue_bridge, 1 hand, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 624/624 [00:24<00:00, 25.12it/s]


In [27]:
filename = '/content/yolo_block/data/video_data/output/video_out.mp4'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 30
frameSize = (640,640)
isColor = True
out = cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor)
for out_img in out_img_list:
    # print(out_img)
    out.write(out_img)

out.release()